In [1]:
from pathlib import Path
from IPython.display import HTML, display
css = Path("../../../css/rtl.css").read_text(encoding="utf-8")
display(HTML(f"<style>{css}</style>"))

# فصل ۱ — مقدمه‌ای بر یادگیری ماشین
# درس ۱۰: پارادایم‌های یادگیری و طبقه‌بندی وظایف (رگرسیون، دسته‌بندی، رتبه‌بندی، پیش‌بینی)


# فصل ۱ — مقدمه‌ای بر یادگیری ماشین
درس ۱۰: پارادایم‌های یادگیری و طبقه‌بندی وظایف (رگرسیون، دسته‌بندی، رتبه‌بندی، پیش‌بینی)


این نوت‌بوک بخشی از یک دوره‌ی دوزبانه و مبتنی بر ژوپیتر است.

### چرا این درس مهم است؟

در عمل، خیلی‌ها از الگوریتم شروع می‌کنند («XGBoost خوب است یا نه؟») قبل از اینکه **نوع وظیفه** مشخص شود.
این ترتیب معمولاً غلط است. تعریف درست وظیفه تعیین می‌کند:

- هدف ($y$) چیست،
- چه فرض‌هایی درباره‌ی تولید داده منطقی است (IID یا وابسته به زمان)،
- چه معیارهایی گزارش می‌شود،
- و چه پروتکل اعتبارسنجی معتبر است.

### دامنه‌ی درس

روی خانواده‌های بسیار رایج در یادگیری ماشین کلاسیک تمرکز داریم:

1. رگرسیون
2. دسته‌بندی (دودویی و چندکلاسه)
3. رتبه‌بندی / یادگیری برای رتبه‌بندی
4. پیش‌بینی (یادگیری نظارت‌شده با وابستگی زمانی)

همچنین ارتباط آن‌ها را با پارادایم‌های یادگیری (نظارت‌شده/بدون‌نظارت/نیمه‌نظارت/تقویتی) بیان می‌کنیم،
اما بخش عملی عمدتاً نظارت‌شده است.

---

## اهداف یادگیری

در پایان درس باید بتوانید:

1. یک سؤال کاربردی را به نوع وظیفه (رگرسیون/دسته‌بندی/رتبه‌بندی/پیش‌بینی) نگاشت کنید.
2. برای هر وظیفه، **زیان‌ها** و **معیارهای** مناسب را انتخاب کنید.
3. تفاوت ارزیابی IID با ارزیابی سری‌زمانی را تشخیص دهید.
4. baseline حداقلی و صحیح در scikit-learn بسازید و نتایج را تفسیر کنید.
5. خطاهای رایج (عدم تطابق معیار، leakage، split نامعتبر) را توضیح دهید.

---

## مدل ذهنی: «وظیفه = نوع هدف + پروتکل ارزیابی + هزینه‌ی تصمیم»

یک تعریف کاربردی از وظیفه:

$$
\text{Task} = \big(\text{target type}, \; \text{valid evaluation}, \; \text{decision cost}\big)
$$

اگر فقط نوع هدف را مشخص کنید و ارزیابی را نادیده بگیرید، احتمالاً مدلی می‌سازید که در تولید شکست می‌خورد.


In [2]:
import io
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, f1_score, classification_report,
    confusion_matrix, roc_auc_score
)
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn.ensemble import RandomForestRegressor

np.random.seed(0)

def read_csv_or_sample(path, sample_csv_text, **kwargs):
    """Read a CSV from repo path; fall back to embedded sample rows if missing."""
    p = Path(path)
    if p.exists():
        return pd.read_csv(p, **kwargs)
    return pd.read_csv(io.StringIO(sample_csv_text), **kwargs)

def synthesize_from_sample(df, n=500, noise=0.05, seed=42):
    """Bootstrap + jitter to make a larger dataset for demonstrative modeling."""
    rng = np.random.default_rng(seed)
    df_big = df.sample(n=n, replace=True, random_state=seed).reset_index(drop=True)
    for col in df_big.columns:
        if pd.api.types.is_numeric_dtype(df_big[col]):
            std = df_big[col].std(ddof=0)
            if np.isfinite(std) and std > 0:
                df_big[col] = df_big[col] + rng.normal(0, noise*std, size=len(df_big))
    return df_big

def synthesize_earthquake_timeseries(df_small, days=220, avg_events_per_day=4, seed=19):
    """Create a multi-day series when only sample rows are available."""
    rng = np.random.default_rng(seed)
    df_small = df_small.copy()
    df_small["timestamp"] = pd.to_datetime(df_small["date"].astype(str) + " " + df_small["time"].astype(str), errors="coerce")
    df_small = df_small.dropna(subset=["timestamp"])

    start = df_small["timestamp"].min().normalize()
    rows = []
    for d in range(days):
        day = start + pd.Timedelta(days=d)
        m = rng.poisson(avg_events_per_day) + 1
        sample = df_small.sample(n=m, replace=True, random_state=int(seed + d)).reset_index(drop=True)
        secs = rng.integers(0, 24*3600, size=m)
        sample["timestamp"] = day + pd.to_timedelta(secs, unit="s")

        for col in ["latitude", "longitude", "depth", "magnitude"]:
            if col in sample.columns:
                sample[col] = pd.to_numeric(sample[col], errors="coerce")
                std = np.nanstd(sample[col])
                if np.isfinite(std) and std > 0:
                    sample[col] = sample[col] + rng.normal(0, 0.10*std, size=m)

        rows.append(sample)

    out = pd.concat(rows, ignore_index=True)
    out["date"] = out["timestamp"].dt.date.astype(str)
    out["time"] = out["timestamp"].dt.time.astype(str)
    return out.drop(columns=["timestamp"])

def rmse(y_true, y_pred):
    """Compute RMSE without using deprecated sklearn squared=... parameter."""
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

print("Setup complete. Versions:")
import sklearn
print("  pandas:", pd.__version__)
print("  numpy:", np.__version__)
print("  sklearn:", sklearn.__version__)


Setup complete. Versions:
  pandas: 2.2.3
  numpy: 2.1.2
  sklearn: 1.5.2


## ۱. پارادایم‌های یادگیری: نظارت از کجا می‌آید؟

**پارادایم یادگیری** مشخص می‌کند مدل چگونه اطلاعات لازم برای تعریف «رفتار خوب» را به‌دست می‌آورد.

### ۱.۱ یادگیری نظارت‌شده

جفت‌های برچسب‌دار $(x_i, y_i)$ را مشاهده می‌کنید و نگاشت $f: x \mapsto y$ را می‌آموزید.

هدف رایج:

$$
\hat{f} = \arg\min_{f \in \mathcal{F}} \; \frac{1}{n} \sum_{i=1}^n \ell\big(y_i, f(x_i)\big) + \lambda \Omega(f)
$$

- $\ell$ تابع زیان است (چقدر پیش‌بینی اشتباه است)
- $\Omega$ منظم‌ساز است (کنترل ظرفیت)
- $\lambda$ شدت منظم‌سازی را تنظیم می‌کند

### ۱.۲ یادگیری بدون‌نظارت

بدون هدف صریح، فقط $x_i$ را می‌بینید و ساختار را یاد می‌گیرید: خوشه‌ها، چگالی‌ها، embeddingها.
در عمل برای نمایش‌سازی، سگمنتیشن و تشخیص ناهنجاری بسیار استفاده می‌شود.

### ۱.۳ نیمه‌نظارت‌شده و نظارت ضعیف

یک مجموعه‌ی کوچک برچسب‌دار و یک مجموعه‌ی بزرگ بدون برچسب دارید.
ممکن است از pseudo-label و ساختار گراف برای انتشار برچسب استفاده کنید.
در «نظارت ضعیف»، برچسب‌های نویزی از قوانین اکتشافی ساخته می‌شود.

### ۱.۴ یادگیری تقویتی (RL) و باندیت‌ها

با محیط تعامل می‌کنید، عمل انتخاب می‌کنید و پاداش می‌گیرید.
تمرکز این درس نیست، اما برخی مسائل رتبه‌بندی/پیشنهاددهی را می‌توان به شکل contextual bandit دید.

---

## ۲. طبقه‌بندی وظایف: نوع هدف و پروتکل ارزیابی

یک طبقه‌بندی فشرده برای وظایف رایج نظارت‌شده:

| نوع وظیفه | هدف $y$ | خروجی پیش‌بینی | معیار رایج |
|---|---|---|---|
| رگرسیون | $\mathbb{R}$ | مقدار | MAE، RMSE، $R^2$ |
| دسته‌بندی دودویی | $\{0,1\}$ | برچسب/احتمال | F1، ROC AUC، PR AUC |
| دسته‌بندی چندکلاسه | $\{1,\dots,K\}$ | احتمال کلاس‌ها | accuracy، macro-F1 |
| رتبه‌بندی | ارتباط/ترجیح زوجی | ترتیب | NDCG@k، MAP، Recall@k |
| پیش‌بینی | $y_{t+h}$ | مقدار آینده | MAE/RMSE بر اساس افق |

نکته: **پیش‌بینی** معمولاً نظارت‌شده است اما **IID نیست**.
پس معنای split آموزش/آزمون متفاوت می‌شود.

---

## ۲.۱ هزینه، آستانه و قانون تصمیم

حتی برای یک نوع وظیفه‌ی ثابت، هزینه‌ها می‌توانند قانون تصمیم را تغییر دهند.

برای دسته‌بندی دودویی:

$$
\hat{y} = \mathbf{1}\big[\; P(y=1\mid x) \ge \tau \;\big]
$$

آستانه‌ی $\tau$ باید بر اساس هزینه‌ی خطای نوع اول/دوم تعیین شود، نه صرفاً $0.5$.

برای رتبه‌بندی، معمولاً ابتدای لیست مهم است چون ظرفیت محدود دارید:

- فقط به ۱٪ بالای leadها ایمیل می‌زنید
- فقط ۱۰ محصول اول را نمایش می‌دهید
- فقط ۵۰ تراکنش اول را بررسی می‌کنید

پس ارزیابی باید با معیارهای top-$k$ انجام شود.

---

## ۲.۲ چک‌لیست کوچک برای فرموله‌کردن وظیفه

قبل از آموزش هر چیزی، این‌ها را بنویسید:

1. هدف $y$ چیست (با واحد/تعریف دقیق)؟
2. بعد از پیش‌بینی چه اقدام/تصمیمی انجام می‌شود؟
3. هزینه‌ی خطاها چیست؟
4. قانون split داده باید چه باشد تا شبیه تولید شود؟

حالا این ایده‌ها را با مثال‌های کدنویسی ملموس می‌کنیم.


## ۳. فرموله‌کردن کوچک: رگرسیون در برابر دسته‌بندی و رتبه‌بندی

این مثال کوچک یک ویژگی $x$ و یک خروجی پیوسته را در نظر می‌گیرد.

- در **رگرسیون**، $y$ را مستقیم پیش‌بینی می‌کنید.
- در **دسته‌بندی**، پیش‌بینی می‌کنید آیا $y$ از یک آستانه بیشتر است یا نه.
- در **رتبه‌بندی**، از امتیاز پیش‌بینی برای مرتب‌سازی نمونه‌ها بر اساس ریسک استفاده می‌کنید.

با اینکه داده‌ی خام یکسان است، پارامترها و ارزیابی متفاوت خواهد بود.


In [3]:
# A tiny illustration: the same data can be framed as regression or classification.

rng = np.random.default_rng(0)
n = 12
x = rng.normal(size=n)
y_cont = 2*x + rng.normal(scale=0.5, size=n)          # continuous target
y_bin = (y_cont > np.median(y_cont)).astype(int)      # binarized target

toy = pd.DataFrame({"x": x, "y_cont": y_cont, "y_bin": y_bin})
display(toy)

# Regression framing
reg = LinearRegression().fit(toy[["x"]], toy["y_cont"])
y_hat_reg = reg.predict(toy[["x"]])

# Classification framing
clf = LogisticRegression().fit(toy[["x"]], toy["y_bin"])
p_hat = clf.predict_proba(toy[["x"]])[:, 1]

print("Regression coefficients:", reg.coef_[0], "intercept:", reg.intercept_)
print("Classification coefficient:", clf.coef_[0,0], "intercept:", clf.intercept_[0])
print("\nFirst 5 predictions:")
for i in range(5):
    print(f"  x={x[i]: .3f}  y_cont={y_cont[i]: .3f}  y_hat_reg={y_hat_reg[i]: .3f}  p(y=1)={p_hat[i]: .3f}")


,x,y_cont,y_bin
0,0.125730,-0.911055,0
1,-0.132105,-0.373606,0
2,0.640423,0.657890,1
3,0.104900,-0.156333,1
4,-0.535669,-1.343468,0
5,0.361595,0.565040,1
6,1.304000,2.813815,1
7,0.947081,2.415419,1
8,-0.703735,-1.471738,0
9,-1.265421,-1.847611,0


Regression coefficients: 1.9512520123535053 intercept: -0.12409791654978095
Classification coefficient: 1.4488474737404589 intercept: -0.027935787302401996

First 5 predictions:
  x= 0.126  y_cont=-0.911  y_hat_reg= 0.121  p(y=1)= 0.538
  x=-0.132  y_cont=-0.374  y_hat_reg=-0.382  p(y=1)= 0.445
  x= 0.640  y_cont= 0.658  y_hat_reg= 1.126  p(y=1)= 0.711
  x= 0.105  y_cont=-0.156  y_hat_reg= 0.081  p(y=1)= 0.531
  x=-0.536  y_cont=-1.343  y_hat_reg=-1.169  p(y=1)= 0.309


## ۴. مثال رگرسیون: پیش‌بینی قیمت خانه

### ۴.۱ صورت مسئله

با داشتن ویژگی‌های یک خانه، قیمت فروش را پیش‌بینی کنید:

- ورودی‌ها $x$: متراژ، تعداد اتاق‌ها، محله و ...
- هدف $y$: قیمت (واحد پول)

این یک وظیفه‌ی رگرسیون است چون $y \in \mathbb{R}$.

### ۴.۲ رویکرد baseline

می‌سازیم:

1. پایپ‌لاین پیش‌پردازش:
   - عددی: جایگذاری میانه + استانداردسازی
   - دسته‌ای: جایگذاری پرتکرار + one-hot encoding
2. مدل رگرسیون خطی

### ۴.۳ معیارها

MAE، RMSE و $R^2$ را گزارش می‌کنیم:

- MAE در واحد هدف قابل تفسیر است.
- RMSE خطاهای بزرگ را بیشتر جریمه می‌کند.
- $R^2$ برای «واریانس توضیح‌داده‌شده» مفید است اما بین دیتاست‌ها قابل مقایسه‌ی مستقیم نیست.

baseline را پیاده‌سازی می‌کنیم.


In [4]:
# Regression dataset: house-prices
HOUSE_PATH = "../../../Datasets/Regression/house-prices.csv"
sample_house = "Home,Price,SqFt,Bedrooms,Bathrooms,Offers,Brick,Neighborhood\n1,114300,1790,2,2,2,No,East\n2,114200,2030,4,2,3,No,East\n3,114800,1740,3,2,1,No,East\n4,94700,1980,3,2,3,No,East\n5,119800,2130,3,3,3,No,East\n"

house_df = read_csv_or_sample(HOUSE_PATH, sample_house)
print("Raw rows (head):")
print(house_df.head().to_string(index=False))

df = synthesize_from_sample(house_df, n=500, noise=0.08, seed=13)

y = df["Price"].astype(float)
X = df.drop(columns=["Price"]).copy()

num_cols = ["SqFt", "Bedrooms", "Bathrooms", "Offers"]
cat_cols = ["Brick", "Neighborhood"]

pre = ColumnTransformer([
    ("num", Pipeline([("imp", SimpleImputer(strategy="median")),
                      ("sc", StandardScaler())]), num_cols),
    ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                      ("oh", OneHotEncoder(handle_unknown="ignore"))]), cat_cols),
])

model = Pipeline([("pre", pre), ("linreg", LinearRegression())])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_test)

mae = mean_absolute_error(y_test, pred)
rmse_val = rmse(y_test, pred)
r2 = r2_score(y_test, pred)

print(f"MAE:  {mae:,.2f}")
print(f"RMSE: {rmse_val:,.2f}")
print(f"R^2:  {r2:.3f}")


Raw rows (head):
 Home  Price  SqFt  Bedrooms  Bathrooms  Offers Brick Neighborhood
    1 114300  1790         2          2       2    No         East
    2 114200  2030         4          2       3    No         East
    3 114800  1740         3          2       1    No         East
    4  94700  1980         3          2       3    No         East
    5 119800  2130         3          3       3    No         East
MAE:  7,283.09
RMSE: 9,529.77
R^2:  0.889


## ۵. مثال دسته‌بندی دودویی: پیش‌بینی دیابت

### ۵.۱ صورت مسئله

با داشتن اندازه‌گیری‌های بیمار، پیش‌بینی کنید بیمار دیابتی است یا نه.

- ورودی‌ها $x$: گلوکز، BMI، سن و ...
- هدف $y \in \{0,1\}$: دیابتی / غیردیابتی

### ۵.۲ اهمیت پیش‌بینی احتمالی

بسیاری از تصمیم‌های پایین‌دستی به «ریسک» نیاز دارند نه فقط برچسب قطعی.
رگرسیون لجستیک احتمال $P(y=1\mid x)$ را برآورد می‌کند.

### ۵.۳ معیارها

محاسبه می‌کنیم:

- Accuracy
- F1
- ROC AUC
- ماتریس درهم‌ریختگی (Confusion Matrix)

در داده‌های نامتوازن، accuracy به‌تنهایی گمراه‌کننده است؛ F1 و AUC سیگنال بیشتری می‌دهند.


In [5]:
# Classification dataset: diabetes
DIAB_PATH = "../../../Datasets/Classification/diabetes.csv"
sample_diabetes = "Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,classification\n6,148,72,35,0,33.6,0.627,50,Diabetic\n1,85,66,29,0,26.6,0.351,31,Non-Diabetic\n8,183,64,0,0,23.3,0.672,32,Diabetic\n1,89,66,23,94,28.1,0.167,21,Non-Diabetic\n0,137,40,35,168,43.1,2.288,33,Diabetic\n"

diab_df = read_csv_or_sample(DIAB_PATH, sample_diabetes)
print("Raw rows (head):")
print(diab_df.head().to_string(index=False))

df = synthesize_from_sample(diab_df, n=600, noise=0.10, seed=7)

y = (df["classification"].astype(str) == "Diabetic").astype(int)
X = df.drop(columns=["classification"])

pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc", StandardScaler()),
    ("logreg", LogisticRegression(max_iter=1000, random_state=0)),
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0, stratify=y)

pipe.fit(X_train, y_train)
proba = pipe.predict_proba(X_test)[:, 1]
pred = (proba >= 0.5).astype(int)

acc = accuracy_score(y_test, pred)
f1 = f1_score(y_test, pred)
auc = roc_auc_score(y_test, proba)
cm = confusion_matrix(y_test, pred)

print(f"Accuracy: {acc:.3f}")
print(f"F1:       {f1:.3f}")
print(f"ROC AUC:  {auc:.3f}")
print("Confusion matrix [[TN, FP], [FN, TP]]:")
print(cm)


Raw rows (head):
 Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin  BMI  DiabetesPedigreeFunction  Age classification
           6      148             72             35        0 33.6                     0.627   50       Diabetic
           1       85             66             29        0 26.6                     0.351   31   Non-Diabetic
           8      183             64              0        0 23.3                     0.672   32       Diabetic
           1       89             66             23       94 28.1                     0.167   21   Non-Diabetic
           0      137             40             35      168 43.1                     2.288   33       Diabetic
Accuracy: 0.793
F1:       0.667
ROC AUC:  0.828
Confusion matrix [[TN, FP], [FN, TP]]:
[[88  8]
 [23 31]]


## ۶. دسته‌بندی چندکلاسه: Iris

در دسته‌بندی چندکلاسه، بین $K>2$ کلاس پیش‌بینی می‌کنید.
حتی اگر accuracy بالا باشد، بهتر است عملکرد هر کلاس را در گزارش دسته‌بندی بررسی کنید.

از دیتاست iris برای نمایش استفاده می‌کنیم.


In [6]:
# Multi-class dataset: iris
IRIS_PATH = "../../../Datasets/Classification/iris.csv"
sample_iris = "sepal_length,sepal_width,petal_length,petal_width,classification\n5.1,3.5,1.4,0.2,Iris-setosa\n4.9,3.0,1.4,0.2,Iris-setosa\n5.0,3.6,1.4,0.2,Iris-setosa\n7.0,3.2,4.7,1.4,Iris-versicolor\n6.4,3.2,4.5,1.5,Iris-versicolor\n6.9,3.1,4.9,1.5,Iris-versicolor\n6.3,3.3,6.0,2.5,Iris-virginica\n5.8,2.7,5.1,1.9,Iris-virginica\n7.1,3.0,5.9,2.1,Iris-virginica\n"

iris_df = read_csv_or_sample(IRIS_PATH, sample_iris)
print("Raw rows (head):")
print(iris_df.head().to_string(index=False))

df = synthesize_from_sample(iris_df, n=450, noise=0.08, seed=11)

y = df["classification"].astype(str)
X = df.drop(columns=["classification"])

pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc", StandardScaler()),
    ("logreg", LogisticRegression(max_iter=1000, random_state=0)),
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0, stratify=y)

pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)

acc = accuracy_score(y_test, pred)
print(f"Accuracy (multi-class): {acc:.3f}")
print("\nClassification report:")
print(classification_report(y_test, pred, zero_division=0))


Raw rows (head):
 sepal_length  sepal_width  petal_length  petal_width classification
          5.4          3.7           1.5          0.2    Iris-setosa
          4.8          3.4           1.6          0.2    Iris-setosa
          4.8          3.0           1.4          0.1    Iris-setosa
          4.3          3.0           1.1          0.1    Iris-setosa
          5.8          4.0           1.2          0.2    Iris-setosa
Accuracy (multi-class): 0.948

Classification report:
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        42
Iris-versicolor       0.97      0.86      0.92        44
 Iris-virginica       0.89      0.98      0.93        49

       accuracy                           0.95       135
      macro avg       0.95      0.95      0.95       135
   weighted avg       0.95      0.95      0.95       135



## ۷. baseline رتبه‌بندی / یادگیری برای رتبه‌بندی

### ۷.۱ چه چیزی رتبه‌بندی را متفاوت می‌کند؟

در دسته‌بندی، هر نمونه مستقل است و یک برچسب دارد.

در رتبه‌بندی، معمولاً یک «پرسش/کانتکست» (کاربر، سشن، جستجو) و مجموعه‌ای از آیتم‌های کاندید دارید.
باید آیتم‌ها را طوری مرتب کنید که سودمندی در ابتدای لیست بیشینه شود.

### ۷.۲ pointwise در برابر pairwise و listwise

- **Pointwise:** برای هر آیتم یک امتیاز ارتباط پیش‌بینی کن و سپس sort کن
- **Pairwise:** ترجیح زوجی یاد بگیر (A باید بالاتر از B باشد)
- **Listwise:** یک هدف سطح-لیست را مستقیم بهینه کن

ما یک baseline pointwise پیاده‌سازی می‌کنیم چون ساده است و به‌عنوان قدم اول رایج است.

### ۷.۳ ارزیابی با NDCG

NDCG به ابتدای لیست وزن بیشتری می‌دهد:

$$
\mathrm{DCG}@k = \sum_{i=1}^{k} \frac{2^{rel_i}-1}{\log_2(i+1)}, \quad
\mathrm{NDCG}@k = \frac{\mathrm{DCG}@k}{\mathrm{IDCG}@k}
$$

یک برچسب ارتباط مصنوعی از ویژگی‌ها می‌سازیم و NDCG@10 و NDCG@50 را محاسبه می‌کنیم.


In [7]:
# Ranking dataset: listings (Airbnb-style)
LIST_PATH = "../../../Datasets/Regression/listings.csv"
sample_listings = "id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license\n13913,Holiday London DB Room Let-on going,54730,Alina,,Islington,51.56861,-0.1127,Private room,57,1,51,2025-02-09,0.29,3,344,10,\n15400,Bright Chelsea  Apartment. Chelsea!,60302,Philippa,,Kensington and Chelsea,51.4878,-0.16813,Entire home/apt,180,4,96,2024-04-28,0.52,1,11,2,\n17402,Very Central Modern 3-Bed/2 Bath By Oxford St W1,67564,Liz,,Westminster,51.52195,-0.14094,Entire home/apt,510,3,56,2024-02-19,0.33,5,293,0,\n24328,Battersea live/work artist house,41759,Joe,,Wandsworth,51.47072,-0.16266,Entire home/apt,213,90,94,2022-07-19,0.54,1,194,0,\n31036,Bright  compact 1 Bedroom Apartment Brick Lane,133271,Hendryks,,Tower Hamlets,51.52425,-0.06997,Entire home/apt,100,2,126,2025-02-20,0.70,8,353,3,\n"

list_df = read_csv_or_sample(LIST_PATH, sample_listings)
print("Raw rows (head):")
print(list_df.head().to_string(index=False))

df = synthesize_from_sample(list_df, n=800, noise=0.06, seed=17)

# Convert and clip numeric columns to keep them in reasonable ranges (prevents invalid log1p)
num_cols_all = ["price", "minimum_nights", "number_of_reviews", "reviews_per_month",
                "calculated_host_listings_count", "availability_365"]
for c in num_cols_all:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df["price"] = df["price"].fillna(df["price"].median()).clip(lower=0)
df["reviews_per_month"] = df["reviews_per_month"].fillna(0.0).clip(lower=0)
df["number_of_reviews"] = df["number_of_reviews"].fillna(0.0).clip(lower=0)
df["availability_365"] = df["availability_365"].fillna(df["availability_365"].median()).clip(lower=0)
df["minimum_nights"] = df["minimum_nights"].fillna(df["minimum_nights"].median()).clip(lower=1)
df["calculated_host_listings_count"] = df["calculated_host_listings_count"].fillna(1.0).clip(lower=1)

df["room_type"] = df["room_type"].astype(str)
df["neighbourhood"] = df["neighbourhood"].astype(str)

# Synthetic relevance label (for demonstration only)
score = (0.6*np.log1p(df["number_of_reviews"]) + 0.3*np.log1p(df["availability_365"]) - 0.002*df["price"])
qs = score.quantile([0.25, 0.5, 0.75]).values
df["relevance"] = np.digitize(score, qs).astype(int)

def ndcg_at_k(y_true, y_score, k=10):
    y_true = np.asarray(y_true)
    y_score = np.asarray(y_score)
    order = np.argsort(y_score)[::-1][:k]
    gains = (2**y_true[order] - 1)
    discounts = 1.0 / np.log2(np.arange(2, len(order)+2))
    dcg = np.sum(gains * discounts)

    ideal_order = np.argsort(y_true)[::-1][:k]
    ideal_gains = (2**y_true[ideal_order] - 1)
    idcg = np.sum(ideal_gains * discounts)
    return float(dcg / idcg) if idcg > 0 else 0.0

y = df["relevance"].astype(int)
X = df[[
    "price", "minimum_nights", "number_of_reviews", "reviews_per_month",
    "calculated_host_listings_count", "availability_365",
    "room_type", "neighbourhood"
]].copy()

num_cols = ["price", "minimum_nights", "number_of_reviews", "reviews_per_month",
            "calculated_host_listings_count", "availability_365"]
cat_cols = ["room_type", "neighbourhood"]

pre = ColumnTransformer([
    ("num", Pipeline([("imp", SimpleImputer(strategy="median")),
                      ("sc", StandardScaler())]), num_cols),
    ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                      ("oh", OneHotEncoder(handle_unknown="ignore"))]), cat_cols)
])

ranker = Pipeline([
    ("pre", pre),
    ("rf", RandomForestRegressor(n_estimators=200, random_state=0, n_jobs=-1, min_samples_leaf=3))
])

# Hold out entire neighbourhoods (group-aware evaluation)
unique_neigh = df["neighbourhood"].unique()
rng = np.random.default_rng(0)
test_neigh = set(rng.choice(unique_neigh, size=max(1, len(unique_neigh)//4), replace=False))
test_mask = df["neighbourhood"].isin(test_neigh)

X_train, X_test = X[~test_mask], X[test_mask]
y_train, y_test = y[~test_mask], y[test_mask]

ranker.fit(X_train, y_train)
y_score = ranker.predict(X_test)

print("NDCG@10:", round(ndcg_at_k(y_test, y_score, k=10), 3))
print("NDCG@50:", round(ndcg_at_k(y_test, y_score, k=50), 3))

top_idx = np.argsort(y_score)[::-1][:10]
ranked = df.loc[X_test.index[top_idx], ["name","neighbourhood","room_type","price","number_of_reviews","availability_365","relevance"]]
print("\nTop-10 ranked items (sample):")
print(ranked.reset_index(drop=True).to_string(index=False))


Raw rows (head):
   id                                             name  host_id host_name  neighbourhood_group          neighbourhood  latitude  longitude       room_type  price  minimum_nights  number_of_reviews last_review  reviews_per_month  calculated_host_listings_count  availability_365  number_of_reviews_ltm  license
13913              Holiday London DB Room Let-on going    54730     Alina                  NaN              Islington  51.56861   -0.11270    Private room   57.0               1                 51  2025-02-09               0.29                               3               344                     10      NaN
15400              Bright Chelsea  Apartment. Chelsea!    60302  Philippa                  NaN Kensington and Chelsea  51.48780   -0.16813 Entire home/apt    NaN               4                 96  2024-04-28               0.52                               1                11                      2      NaN
17402 Very Central Modern 3-Bed/2 Bath By Oxford St W

## ۸. مثال پیش‌بینی: بزرگی زلزله در سطح روزانه

### ۸.۱ پیش‌بینی نظارت‌شده است اما وابسته به زمان

پیش‌بینی $y_{t+h}$ را با استفاده از اطلاعات تا زمان $t$ انجام می‌دهد.
با اینکه نظارت‌شده است، وابستگی زمانی قوانین را عوض می‌کند:

- نباید از داده‌ی آینده برای آموزش استفاده کنید.
- اعتبارسنجی باید بر اساس split زمانی باشد.

### ۸.۲ مهندسی ویژگی با lag

یک روش ساده این است که داده را روزانه تجمیع کنیم و ویژگی‌های lag بسازیم:

- میانگین lag‌شده‌ی بزرگی
- تعداد رخدادهای lag‌شده

سپس یک مدل رگرسیون آموزش دهیم.

### ۸.۳ ارزیابی لغزان

از `TimeSeriesSplit` برای شبیه‌سازی ارزیابی پنجره‌ای استفاده می‌کنیم.
این کامل نیست (در سیستم واقعی ممکن است gap داشته باشید)، اما از shuffle بسیار امن‌تر است.

پیاده‌سازی می‌کنیم.


In [8]:
# Forecasting dataset: earthquake
EQ_PATH = "../../../Datasets/Regression/earthquake.csv"
sample_earthquake = "date,time,latitude,longitude,depth,magnitude\n2008-11-01,00:31:25,-0.6,98.89553,20.0,2.99\n2008-11-02,01:34:29,-6.61,129.38722,30.1,5.51\n2008-11-03,01:38:14,-3.65,127.99068,5.0,3.54\n2008-11-04,02:20:05,-4.2,128.097,5.0,2.42\n2008-11-05,02:32:18,-4.09,128.20047,10.0,2.41\n"

eq_df = read_csv_or_sample(EQ_PATH, sample_earthquake)
print("Raw rows (head):")
print(eq_df.head().to_string(index=False))

# If you only have a few sample rows, create a multi-day synthetic series for demonstration.
if len(eq_df) < 100:
    df = synthesize_earthquake_timeseries(eq_df, days=220, avg_events_per_day=4, seed=19)
else:
    df = eq_df.copy()

df["timestamp"] = pd.to_datetime(df["date"].astype(str) + " " + df["time"].astype(str), errors="coerce")
df = df.dropna(subset=["timestamp"]).sort_values("timestamp")

daily = df.set_index("timestamp").resample("D")["magnitude"].agg(["mean", "count"]).reset_index()
daily.columns = ["date", "mean_magnitude", "event_count"]

for lag in [1, 2, 3, 7]:
    daily[f"lag_mean_{lag}"] = daily["mean_magnitude"].shift(lag)
    daily[f"lag_count_{lag}"] = daily["event_count"].shift(lag)

daily = daily.dropna().reset_index(drop=True)

y = daily["mean_magnitude"]
X = daily.drop(columns=["date", "mean_magnitude"])

tscv = TimeSeriesSplit(n_splits=5)
model = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc", StandardScaler()),
    ("ridge", Ridge(alpha=1.0, random_state=0)),
])

maes = []
for train_idx, test_idx in tscv.split(X):
    model.fit(X.iloc[train_idx], y.iloc[train_idx])
    pred = model.predict(X.iloc[test_idx])
    maes.append(mean_absolute_error(y.iloc[test_idx], pred))

print("TimeSeries CV MAE (mean):", round(float(np.mean(maes)), 4))
print("TimeSeries CV MAE (std): ", round(float(np.std(maes)), 4))

model.fit(X, y)
daily["pred_next_mean_mag"] = model.predict(X)

print("\nLast 5 days (actual vs predicted mean magnitude):")
print(daily[["date", "mean_magnitude", "pred_next_mean_mag", "event_count"]].tail(5).to_string(index=False))


Raw rows (head):
      date     time  latitude  longitude  depth  magnitude
2008-11-01 00:31:25     -0.60   98.89553   20.0       2.99
2008-11-01 01:34:29     -6.61  129.38722   30.1       5.51
2008-11-01 01:38:14     -3.65  127.99068    5.0       3.54
2008-11-01 02:20:05     -4.20  128.09700    5.0       2.42
2008-11-01 02:32:18     -4.09  128.20047   10.0       2.41
TimeSeries CV MAE (mean): 0.2143
TimeSeries CV MAE (std):  0.0214

Last 5 days (actual vs predicted mean magnitude):
      date  mean_magnitude  pred_next_mean_mag  event_count
2022-09-22        3.388400            3.467577           25
2022-09-23        3.507105            3.343235           38
2022-09-24        3.461333            3.539307           15
2022-09-25        3.242609            3.466240           23
2022-09-26        3.645238            3.492252           21


## ۹. جمع‌بندی و نکات کلیدی

1. **از هدف و تصمیم شروع کنید.**  
   الگوریتم‌ها بعداً می‌آیند.

2. **معیارها وابسته به نوع وظیفه‌اند.**  
   - رگرسیون: MAE/RMSE
   - دسته‌بندی: F1/AUC (و در صورت نیاز کالیبراسیون)
   - رتبه‌بندی: NDCG@k و معیارهای top-$k$
   - پیش‌بینی: خطاهای وابسته به افق با split زمانی

3. **پروتکل اعتبارسنجی بخشی از تعریف وظیفه است.**  
   سری‌زمانی نیازمند ارزیابی زمان‌محور است.

4. **baseline حیاتی است.**  
   یک baseline ساده و درست می‌تواند از مدل پیچیده‌ای که غلط ارزیابی شده بهتر باشد.

---

## ۱۰. تمرین‌ها

1. در بخش رگرسیون، به‌جای مدل خطی از `RandomForestRegressor` استفاده کنید و MAE/RMSE را مقایسه کنید.
2. در بخش دسته‌بندی، آستانه‌ی $\tau$ را طوری تنظیم کنید که F1 روی validation بیشینه شود.
3. معیار Precision@k را برای بخش رتبه‌بندی پیاده‌سازی کنید و با NDCG@k مقایسه کنید.
4. برای پیش‌بینی، split پنجره‌ی گسترش‌یابنده را امتحان کنید و خطاها را مقایسه کنید.

---

اگر دوست دارید، می‌توانیم در انتهای این نوت‌بوک یک «مینی‌پروژه» اضافه کنیم:
با یک سناریوی کاربردی، نوع وظیفه را تعیین کنید، $y$ را تعریف کنید،
معیار را انتخاب کنید و پایپ‌لاین حداقلی را بنویسید.
